In [ ]:
# Игра "Наборщик"
# Кто сможет составить наибольшее количество слов из букв заданного длинного слова 
# (количество букв имеет значение, а порядок нет)
# Пример: олицетворение - лицо, творение, тело, вор, ...
# Предлагается выводить слово, указывать общее количество скрытых в слове слов, и предлагать поиграть с друзьями в эту игру.
# Можно на "последней странице" привести список всех слов, которые были загаданы, для тех, кто захочет себя проверить.
# Ответы предоставляются (и просто так (обычные слова) и в виде анаграм (в словах перемешаны буквы), как подсказка к загадке)


# vocabularyfile - cловарь - текстовый документ, одно слово в строке, utf8
# word - конкретное слово для игры или wordfrom - минимальная длинна основного слова
# minlen - минимальная длинна искомых слов

# результат:
# word: основное слово
# answer: слова найденные в данном
# clue: подсказки (анаграммы) для всех найденных слов 

# вместе с кодом предоставляются словари существительных для русского и английского языка 
# (только распространенные или все)

In [ ]:
def _if(c,t,f):
    return t if c else f

import traceback

def format_exception(e):
    return str(e)+'\n'+ str(traceback.format_tb(e.__traceback__))

def ensure_dir_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
import codecs
import json
import random
from collections import defaultdict

class WordUtils:
    
    @staticmethod
    def get_frequency(word):
        freq = defaultdict(int)
        for char in word:
            freq[char]+=1
        return freq
    
    @staticmethod
    def get_anagram(word):
        return ''.join(random.sample(word, len(word)))
    
    def get_hangman(word, n):
        chars = list(word)
        for i in random.sample(range(len(word)), min(n,len(word))):
            chars[i] = '*'
        return ''.join(chars)
    
    @staticmethod
    def why_not_composable(innerword, fromword):
        fromfreq = WordUtils.get_frequency(fromword)
        innerfreq = WordUtils.get_frequency(innerword)
        for c,count in innerfreq.items():
            if(fromfreq[c] == 0):
                return ('no_char', str(c))
            if(fromfreq[c] < count):
                return ('too_many_char', str(c))
        return None
    
    @staticmethod
    def is_composable(innerword, fromword):
            return WordUtils.why_not_composable(innerword, fromword) is None
        

class Vocabulary:
    def __init__(self, words):
        if isinstance(words, dict):
            self.words = words
        else:
            self.words = set(words)
            
    @staticmethod
    def read_from(vocabularyfile, encoding='utf8'):
        words = []
        with codecs.open(vocabularyfile, 'r', encoding) as f:
            words = [line.strip() for line in f]
        return Vocabulary(words)
    
    @staticmethod
    def read_multiple_definitions_from(vocabularyfile, encoding='utf8', sep='|', saveas = lambda line: line[1:]):
        #'''save all definitions for one word as list'''
        definitions = defaultdict(list)
        with codecs.open(vocabularyfile, 'r', encoding) as f:
            for line in f:
                line = line.strip().split(sep)
                if len(line) > 0:
                    definitions[line[0].strip()].append(saveas(line))
        return Vocabulary(dict(definitions))
    
    @staticmethod
    def read_definitions_from(vocabularyfile, encoding='utf8', sep='|', saveas = lambda line: line[1:]):
        #'''save only last definition for one word'''
        definitions = dict()
        with codecs.open(vocabularyfile, 'r', encoding) as f:
            for line in f:
                line = line.strip().split(sep)
                if len(line) > 0:
                    definitions[line[0].strip()] = saveas(line)
        return Vocabulary(definitions)
     
    def __contains__(self, word):
        return word in self.words
    
    def __getitem__(self, word):
        return self.words[word]
    
    def __iter__(self):
        return iter(self.words)
    
    def get_randword(self, condition=lambda w: True):
        appropriate = [w for w in self.words if condition(w)]
        return random.choice(appropriate)

    def get_composable_words(self, word):
        freq = WordUtils.get_frequency(word)
        innerwords = set()
        for w in self.words:
            wfreq = WordUtils.get_frequency(w)
            if all(freq[char] >= count for (char,count) in wfreq.items()):
                innerwords.add(w)
        innerwords.discard(word)
        return list(sorted(innerwords))


In [ ]:
nouns = Vocabulary.read_definitions_from(r'locale\ru\vocab\noun.txt', saveas=lambda line: int(line[1]))
allwords = Vocabulary.read_from(r'locale\ru\vocab\all.txt')
defs = Vocabulary.read_multiple_definitions_from(r'locale\ru\vocab\ozhegov.hint.txt', saveas=lambda line: line[1])

In [ ]:
print(u'слово' in nouns)
print(u'варежка' in nouns)
print(u'ботинок' in nouns)
print(u'валенок' in nouns)
print(u'blabla' in defs)
print(u'трон' in allwords)
print(u'трон' in nouns)
print(defs[u'коса'])


hasdef = [w for w in defs.words if w in nouns]
print(len(hasdef))


In [ ]:
import re
import random
import json

class Speaker:
    def __init__(self, phrases, counted={}):
        self.phrases = phrases
        self.counted = counted
        
    def speak(self, about, **kwargs):
        phrase = self.get_phrase(about) 
        for (name,value) in kwargs.items():
            if isinstance(value, int):
                # вы нашли #count# @слов
                phrase = self.speak_count(phrase, name, value)
            phrase = self.speak_conditional(phrase, name, value)   
            phrase = re.sub('#'+name+'#', str(value), phrase)
        return phrase
    
    def speak_conditional(self, phrase, name, value):
        while True:
            m = re.search('\[#'+name+r'#\?(?P<true>[^\]\|]*)\|(?P<false>[^\]\|]*)\]', phrase)
            if m is None:
                break
            replacement = _if(value, m.group('true'), m.group('false'))
            phrase = phrase[:m.start()]+replacement+phrase[m.end():]
        return phrase
    
    def speak_count(self, phrase, name, value):
        while True:
            m = re.search('#'+name+'#(\s+)@(\w+)', phrase)
            if m is None:
                break
            replacement = str(value)+m.group(1)+self.speak_count_of(value, m.group(2))
            phrase = phrase[:m.start()]+replacement+phrase[m.end():]
        return phrase
    
    # 1 слово/очко, 2-4 слова/очка, 0,5-10,11-14,25-100 слов/очков 
    def speak_count_of(self, count, ofobject):
        if ofobject not in self.counted:
            return ofobject
        variants = self.counted[ofobject]
        lastdigit = abs(count)%10
        prevdigit = (abs(count)%100)//10
        
        if prevdigit != 1:
            if lastdigit == 1 :
                return variants[0]
            elif lastdigit <= 4 and lastdigit >= 2 and prevdigit != 1:
                return variants[1]
        return variants[2]        
    
    def get_phrase(self, about):
        if about not in self.phrases:
            return about
        phrases_about = self.phrases[about]
        if len(phrases_about)==0:
            print("No phrases: "+about)
            return about
        phraseind = random.randint(0, len(phrases_about)-1)
        return phrases_about[phraseind]
                                   
                                   
def get_speaker(phrasesfile):
    data = None
    with codecs.open(phrasesfile, 'r', 'utf8') as f:
        data = json.loads(f.read())
    
    return Speaker(data['phrases'], data['counted'])
                

In [ ]:
speaker = get_speaker(r'locale\ru\phrases.json')
print(speaker.speak("no.too_many_char", word = "foo", task = "forest", char="o"))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=0))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=11))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=314))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=121))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=22))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=24))
print(speaker.speak("вы нашли #count# @слов. Поздравляем!", count=37))
print(speaker.speak("вы нашли #count# @слов. [#more10#?Поздравляем!|Маловато как то.]", count=20, more10 = True))
print(speaker.speak("вы нашли #count# @слов. [#more10#?Поздравляем!|Маловато как то.]", count=3, more10 = False))
print(speaker.speak("вы нашли #count# @слов. [#count#?Поздравляем!|Маловато как то.]", count=3))
print(speaker.speak("вы нашли #count# @слов. [#count#?Поздравляем!|Маловато как то.]", count=0))
#print(speaker.phrases)
#print(speaker.counted)


In [ ]:
speaker.speak("Из #total_count# @слов вы уже нашли #count_found# на #score# @очков. Продолжайте в том же духе!", total_count =)

In [ ]:
import re
class CommandParser:
    def __init__(self, commands):
        self.commands = commands
    
    def parse(self, message):
        for (cformat, command) in self.commands:
            if re.search(cformat, message, flags=re.IGNORECASE):
                return command
        return None

def get_command_parser(commandsfile):
    commands = None
    with codecs.open(commandsfile, 'r', 'utf8') as f:
        commands = json.loads(f.read())
    return CommandParser(commands)
            

In [ ]:
command_parser = get_command_parser(r'locale\ru\commands.json')
    
msgs = [u'Давай Играть еще раз!', u'сыграем ещё раз?', u'какой счет?', u'сколько у нас очков?', u'какое было задание?', u'какое слово?', u'скажи ответ на подсказку']
for msg in msgs:
    print(msg)
    print(command_parser.parse(msg))

In [ ]:
class Hint:
    def __init__(self, type_, hint, **kvargs):
        self.type_ = type_
        self.hint = hint
        for (k,v) in kvargs.items():
            self.__dict__[k] = v
    
    def __eq__(self, other):
        if type(other) is type(self):
            return self.type_ == other.type_ and self.hint == other.hint
        return False
    
    def __hash__(self):
        return hash(self.type_) ^ hash(self.hint)
    
    def _export(self):
        return self.__dict__
    @staticmethod
    def _import(data):
        self = Hint(**data)
        return self
        
        
class GameTask:
    def __init__(self, word, composable_words_factors, minlen):
        '''composable_words_factors - all words with their scores
        minlen - minimum word length
        checkword - check the status word in language ('noun', 'word' or 'not exist')
        gethint - return random hint for word (Hint)'''
        self.word = word
        self.composable_words = composable_words_factors
        self.minlen = minlen 
        
class GameTaskFactory:
    def __init__(self, nouns, allwords, defs):
        self.nouns = nouns
        self.allwords = allwords
        self.defs = defs

    def get_hint(self, word):
        #если у слова есть определение с большой вероятностью подсказка будет определением
        if word in self.defs and random.uniform(0,1) < 0.8: 
            word_defs = self.defs[word]
            definition = random.choice(word_defs)
            return Hint('definition', definition)
        if len(word) > 5 and random.uniform(0,1) < 0.6:
            return Hint('hangman', WordUtils.get_hangman(word, (len(word)+1)//2))
        return Hint('anagram', WordUtils.get_anagram(word))
    
    def check_word(self, word):
        if word in self.nouns:
            return 'noun'
        elif word in self.allwords or word in self.defs:
            return 'word'
        return 'not exist'
    
    def get_new_task(self, word= lambda w: len(w) >= 12, minlen=3):
        '''word can be string or function(string)'''
        word = word if isinstance(word, str) else self.nouns.get_randword(word)
        composable_words = [w for w in self.nouns.get_composable_words(word) if len(w) >= minlen]
        scores = {}
        for w in composable_words:
            scores[w] = self.nouns[w]*(len(w) + sum(range(0,max(0,len(w)-3))))
            if w in word:
                scores[w] = len(w)
            
        return GameTask(word, scores, minlen=minlen)

In [ ]:
import re 
from collections import deque
from __future__ import division
import itertools

class PlayerScore:
    def __init__(self, name):
        self.name = name
        self.score = 0
        self.words = 0
        
class HintsCollection:
    def __init__(self, capacity=30):
        self.capacity = capacity
        self.recently_hinted = deque()
        self.hinted = defaultdict(list)
        
    def __len__(self):
        return len(self.recently_hinted)
    
    def _export(self):
        dhinted = {}
        for (w,hlist) in self.hinted.items():
            dhinted[w] = [h._export() for h in hlist]
        return {'hinted':dhinted, 
                'recently_hinted': list(self.recently_hinted),
                'capacity': self.capacity}
    @staticmethod
    def _import(data):
        self = HintsCollection()
        self.hinted = defaultdict(list)
        for (w,hlist) in data["hinted"].items():
            self.hinted[w] = [Hint._import(h) for h in hlist]
        self.recently_hinted = deque(data["recently_hinted"])
        self.capacity = data["capacity"]
        return self
        
    def make_hint(self, word, hint):
        self.hinted[word].append(hint)
        if word in self.recently_hinted:
            self.recently_hinted = [w for w in self.recently_hinted if w != word]
        self.recently_hinted.append(word)
        if len(self.recently_hinted) > 50:
            self.recently_hinted.popleft()
    
    def was_hinted(self, word, hint=None):
        return word in self.hinted and (hint is None or hint in self.hinted[word])
    
    def pop_last_hint(self):
        if len(self.recently_hinted) == 0:
            return None
        word = self.recently_hinted.pop()
        return (word, self.hinted[word][-1])


In [ ]:
hcollection = HintsCollection()
hcollection.make_hint('корова', Hint('definition', 'даёт молоко'))
hcollection.pop_last_hint()
data = hcollection._export()
print(HintsCollection._import(data))

In [ ]:
from collections import deque
import time

class WordGame:
    def __init__(self, task_factory, speaker, commands=None):
        '''get_new_game is function that should return object game with word 
        and dictionary with composable words and theirs score factor'''
        
        self.speaker = speaker
        self.commands = commands
        self.task_factory = task_factory
        
        
        self.game_id = 0
        self.started_at = 0
        self.updated_at = 0
        self.finished_at = 0
        
        self.game = None
        self.status = "NotStarted"
        self.players = dict() #score and words for each player by id
        self.foundwords = dict() #what word was mentioned by what user (None if word was answered by bot)
        self.hints = HintsCollection()
        
        self.message_too_long_was_said = 0
        self.gameover_was_said = 0
        self.previous_games = []
        self.starts = deque(maxlen=15)
        
    
    def _export(self):
        '''exports game state to simple json-serializable view'''
        return {'game_id':self.game_id,
                'started_at':self.started_at,
                'started_at(str)':time.strftime("%Y.%m.%d %H:%M:%S ", time.localtime(self.started_at)),
                'updated_at':self.updated_at,
                'updated_at(str)':time.strftime("%Y.%m.%d %H:%M:%S ", time.localtime(self.updated_at)),
                'finished_at': self.finished_at,
                'finished_at(str)': time.strftime("%Y.%m.%d %H:%M:%S ", time.localtime(self.finished_at)),
                'status': self.status,
                'word':  _if(self.game is not None, self.game.word, None),
                'foundwords':self.foundwords,
                'players': dict((id,player.name) for (id,player) in self.players.items()),
                'hints': self.hints._export()}
    
    def _import(self, data):
        '''imports game state from simple json-serializable view'''
        self.game_id = data['game_id']
        self.started_at = data['started_at']
        self.updated_at = data['updated_at']
        self.finished_at = data['finished_at']
        self.status = data['status']
        word = data['word']
        self.game = _if(word is not None, self.task_factory.get_new_task(word=word), None)
        self.players = dict((int(id), PlayerScore(name)) for (id,name) in data['players'].items())
        self.foundwords = data['foundwords']
        if self.game is not None:
            for (w,player_id) in self.foundwords.items():
                if player_id is None: #bot give answer for hint
                    continue
                self.players[player_id].words += 1
                if w in self.game.composable_words:
                    self.players[player_id].score += self.game.composable_words[w]
        self.hints = HintsCollection._import(data['hints'])
        self.previous_games= []
            
        
    
    def register_player(self, player_id, name):
        if player_id not in self.players:
            self.players[player_id] = PlayerScore(name)
        
    def process(self, player_id, message):
        if self.commands is not None:
            command = self.commands.parse(message)
            if command is not None:
                message = command
        
        if message.startswith('/help'):
            return self.get_help()
        if message.startswith('/newgame') or self.status=="NotStarted":
            return self.start_new_game(message)
        if message.startswith('/start'):
            if self.status=="NotStarted" or self.status=="GameOver":
                return self.start_new_game()
            return self.get_task()
        
            
        if message.startswith("/score"):
            return self.get_score(byplayers=random.uniform(0,1)<0.33)
        if message.startswith("/foundwords"):
            return self.get_score(bywords=True)
        if message.startswith("/done") or len(self.foundwords)==len(self.game.composable_words) or self.status=="GameOver":
            return self.end_game()
        if message.startswith("/task"):
            return self.get_task()
        
        self.updated_at = time.time()
        if message.startswith('/hintanswer'):
            return self.get_hint_answer()
        if message.startswith('/hint'):
            return self.get_hint()
 
        result = self.process_words(player_id, message)
        if len(self.foundwords)==len(self.game.composable_words):
            return self.end_game()
        return result
        
        
    # сообщение + корректное или некорректное слово
    def process_word(self, player_id, word):
        word = word.lower()
        
        
        if word in self.foundwords:
            return {"phrase": self.speaker.speak('no.word_already_said', word=word), "word": word, "correct": True}
        if word in self.game.composable_words:
            wscore = self.game.composable_words[word]
            player_score = self.players[player_id]
            player_score.words += 1
            player_score.score += wscore
            self.foundwords[word] = player_id
            what_to_say = 'ok'
            if wscore > 10 or len(word)  >= 6:
                what_to_say = 'ok.good'
            if wscore > 20:
                what_to_say = 'ok.exellent'
            if word in self.game.word:
                what_to_say = 'ok.cheat'
            
            return {"phrase": self.speaker.speak(what_to_say, word=word, wscore=wscore), 
                    "word":word, "correct":True}
        
        if word == self.game.word:
            return {"phrase": self.speaker.speak('ok.cheat', word=word, wscore=0), 
                    "word":word, "correct":True}
        
        wordstatus = self.task_factory.check_word(word)
        if wordstatus == 'not exist':
            return {"phrase": self.speaker.speak('no.word_not_exist', word=word), "word": word, "correct": False}
        if wordstatus == 'word':
            return {"phrase": self.speaker.speak('no.word_is_not_noun', word=word), "word": word, "correct": False}
        if len(word) < self.game.minlen:
            return {"phrase": self.speaker.speak('no.too_short', word=word), "word": word, "correct": False}
        
        #это существующее слово и оно существительное. возможно,  
        
        why_wrong = WordUtils.why_not_composable(word, self.game.word)
        if why_wrong is not None:
            return {"phrase":self.speaker.speak('no.'+why_wrong[0], word = word, taskword = self.game.word, char=why_wrong[1]), 
                    "word": word, "correct": False}
        else:
            return {"phrase": "FATAL ERROR! YOU BREAK ME COMPLETELY!", "word": word, "correct": False}
        
    def process_words(self, player_id, message):
        message = message.lower()
        words = re.findall(r'[\w-]+', message)
        if (re.search(r'[^\s,\.а-яё-]+', message)):
            return ""
        if (len(words)==0):
            return ""
        
        if len(message) > 150:
            return self.get_message_too_long()
        
        
        if len(words)==1:
            return self.process_word(player_id, words[0])["phrase"]
        
        if (len(self.players) > 1): #многопользовательская игра - человек может переписываться с другими игроками
            nnouns = [self.task_factory.check_word(w) for w in words].count('noun')
            if nnouns/len(words) < 0.5:
                return ""
        
        results = [self.process_word(player_id, w) for w in words]
        ncorrect = len([r for r in results if r["correct"]])
        if ncorrect > 0 or len(results) <= 3:
            return '\n'.join(_if(r["word"] in r["phrase"], r["phrase"], r["word"]+". "+r["phrase"]) for r in results)
        else:
            return self.speaker.speak('no.all_is_wrong')            
    
    def get_message_too_long(self):
        if self.message_too_long_was_said==0 or len(self.players) == 1 or random.randint(0,self.message_too_long_was_said**2) == 0:
            self.message_too_long_was_said += 1
            return self.speaker.speak('message.too_long')
        return ""
    
    def generate_new_task(self, message=None):
        '''try to generate task with parameters'''
        params = {'word':lambda w: len(w) >= 12 and '-' not in w} 
        if message is not None:
            m = re.match(r'/newgame\s+((?P<word>[а-яё]+)|(?P<len>\d+))$', message.strip().lower())
            if m and m.group('word'):
                word = m.group('word')
                params['word'] = word
            elif m and m.group('len'):
                length = int(m.group('len'))
                params['word'] = lambda w: len(w) == length and '-' not in w
        task = None
        try:
            task = self.task_factory.get_new_task(**params)
            if len(task.composable_words)==0:
                task=None
        except Exception as e:
            pass
            
        return task or self.task_factory.get_new_task()
    
    def start_new_game(self, message=None):
        
        if len(self.starts) >= 15 and self.starts[-1] - self.starts[0] < 60 and time.time() - self.starts[-1] < 3*60:
            return self.speaker.speak('new_game.too_many_starts')
        

        task = self.generate_new_task(message)

        if self.game != None:
            self.previous_games.append(self._export())
            
        self.game_id += 1
        self.game = task
        self.players = dict()
        self.foundwords = dict()
        self.hints = HintsCollection()
        self.message_too_long_was_said = 0
        self.gameover_was_said = 0
        self.players = dict()
        self.status = "InGame"
        self.started_at = time.time()
        self.updated_at = time.time()
        self.starts.append(self.started_at)
        return self.speaker.speak('new_game', taskword=self.game.word, count=len(self.game.composable_words))
    
    def end_game(self):
        if self.status != "GameOver":
            self.status = "GameOver"
            self.finished_at = time.time()
            return self.speaker.speak('gameover')+'\n\n'+self.get_score(scoretype="gameover", byplayers=True, bywords=True)
        if self.gameover_was_said == 0 or (self.gameover_was_said < 5 and random.randint(0,self.gameover_was_said**2))==0:
            self.gameover_was_said += 1
            return self.speaker.speak('gameover2')
        return ""
    
    def get_task(self):
        return self.speaker.speak('what_task', taskword=self.game.word, count = len(self.game.composable_words))
    
    def get_score(self, scoretype='ingame', totalscore=True, byplayers=None, bywords=None):
        ingame = scoretype=='ingame'
        score = ""
        ctotal = len(self.game.composable_words)
        cfound = len(self.foundwords)
        crest = ctotal-cfound
        if totalscore:
            score += self.speaker.speak('score.'+scoretype, 
                                        total_count = ctotal, count_found=cfound, count_rest=crest,
                                        taskword = self.game.word,
                                        score=sum([p.score for p in self.players.values()]))
        if byplayers and len(self.players) > 1:
            for player in sorted(self.players.values(), key= lambda p: -p.score):
                score += "\n"+self.speaker.speak('score.player', name=player.name, score=player.score, count=player.words)
        if bywords:
            score += "\n\n"+self.speaker.speak('score.words.'+scoretype)
            toshow = lambda w: ingame == (w in self.foundwords) #if ingame then w in self.foundwords else w not in self.foundwords
            words = [w for w in self.game.composable_words if toshow(w)]
            score += "\n"+self.get_words_score(words, orderbydescscore=ingame)
            
        return score
    
    def get_words_score(self, words, count=(100,160), orderbydescscore=False):
        bylen = lambda w: -len(w)
        stat = []
        
        for (nchar,group) in itertools.groupby(sorted(words, key=bylen), key=bylen):
            nchar=abs(nchar)
            group = sorted(group, 
                           key = lambda w: _if(w in self.game.composable_words, self.game.composable_words[w], 0), 
                           reverse=orderbydescscore)
            ntoshow = count[0]//nchar if len(group) > count[1]//nchar else len(group)
            stat.append(self.speaker.speak('score.words', words=', '.join(group[0:ntoshow]), 
                                           nchar=nchar, nother=len(group)-ntoshow))
        
        return '\n'.join(stat)
        
    
    #hint type: anagram, definition, hangman 
    def get_hint(self):
        notfound = [w for w in self.game.composable_words.keys() if w not in self.foundwords]
        for word in sorted(notfound, 
                          key=lambda w: abs(len(w)-9)+self.game.composable_words[w] + random.randint(0,5) + 
                           (4 if self.hints.was_hinted(w) else 0)):
            count = len([w for w in notfound if WordUtils.is_composable(w, word)])
                 
            hint = self.task_factory.get_hint(word)
            nattempts = 1
            while self.hints.was_hinted(word,hint) and nattempts < 30:
                hint = self.task_factory.get_hint(word)
                nattempts += 1
            if nattempts >= 30:
                continue
                
            self.hints.make_hint(word, hint)
            return self.speaker.speak('hint.'+hint.type_, count=count, **hint.__dict__)
        return ""
    
    def get_hint_answer(self):
        
        if len(self.hints)==0:
            return self.speaker.speak('hint_answer.no_hints')
        (word, hint) = self.hints.pop_last_hint()
        isfound = word in self.foundwords
        if not isfound:
            self.foundwords[word] = None 
        return self.speaker.speak('hint_answer.'+('found' if isfound else 'notfound'), 
                            hint=hint.hint, word=word, wscore=self.game.composable_words[word])
        
    
    def get_help(self):
        '''
        Основные правила:
        Собираем новые слова из букв загаданного слова. Только существительные.

        Основные команды:

        /newgame - начать новую игру
        
        По-русски можно написать что-то вроде:
        сыграем еще раз
        Хочу играть ещё раз!
        задай слово
        Загадай новое слово
        
        /done - завершить игру
        
        я закончил игру
        заверши игру
        
        /task - показать слово из текущей игры
        
        Какое было загадано слово?
        какое задание
        покажи/напиши слово
        Напомни-ка мне, пожалуйста, с каким словом мы играем?

        /score - показать текущий счет
        
        какой счет
        выведи/покажи/напомни счет
        
        /foundwords - показать уже составленные слова
        
        какие слова уже найдены
        какие слова уже нашли
        
        /hint - попросить подсказку
        
        дай подсказку
        
        /hintanswer - попросить ответ на последнюю подсказку
        
        скажи ответ на подсказку
        какой ответ

        /help  - показать данную справку
        
        Покажи справку
        Как играть?
        Что делать?
        '''
        return self.get_help.__doc__
        

In [ ]:
import time
import os.path

class Log:
    def __init__(self, filename):
        self.filename = filename
        ensure_dir_exists(os.path.dirname(filename))
        self.messages = []
        self.lastflush = time.time()
    
    def write(self, message):
        msg = time.strftime("%Y.%m.%d %H:%M:%S ", time.localtime()) + re.sub(r'[\r\n]+', ' ', message.strip())
        self.messages.append(msg)
            
    def flush(self):
        if (len(self.messages) > 0):
            file = codecs.open(self.filename, 'a+', 'utf8')
            for m in self.messages:
                file.write(m+'\n')
            file.close() 
            self.messages = []
        self.lastflush = time.time()

class Saving:
    def __init__(self,filename):
        self.filename = filename
        ensure_dir_exists(os.path.dirname(filename))
        self.lastsaving = 0
    
    def save(self, data):
        file = codecs.open(self.filename, 'w+', 'utf8')
        file.write(json.dumps(data, ensure_ascii=False))
        file.close()
        self.lastsaving = time.time()
        
    def load(self):
        file = codecs.open(self.filename, 'r', 'utf8')
        data = json.loads(file.read())
        file.close()
        return data
    
    def exists(self):
        return os.path.exists(self.filename)

#сохранение игр, в которые уже сыграли. Просто для статистики
class HistorySaving(Log):
    def __init__(self,filename):
        super().__init__(filename)
    
    def write(self, data):
        self.messages.append(json.dumps(data, ensure_ascii=False).strip())
        
        

In [ ]:
import os.path

class Chat:
    
    def __init__(self, chat_id, create_game, datadir = r'data\{0}'):
        d = datadir.format(chat_id)
        if not os.path.exists(d):
            os.makedirs(d)
        
        self.log = Log(os.path.join(d,'log.txt'))
        self.saving = Saving(os.path.join(d, 'save.txt'))
        self.history = HistorySaving(os.path.join(d, 'hist.txt'))
        
        try:
            self.game = create_game()
            game_data = None
            if self.saving.exists():
                game_data = self.saving.load()
                self.game._import(game_data)
        except Exception as e:
            self.log.write("Exception during creating or loading game from saving: "+format_exception(e)+ ' game_data: '+str(game_data))
            self.game = create_game()
    
    def save(self, period=60):
        '''Flush logs, save game and previous games if it is need'''
        if self.game.updated_at - self.saving.lastsaving > period or period==0:
            self.log.write('[info] saving current game')
            self.saving.save(self.game._export())
        if len(self.game.previous_games) > 0:
            self.log.write('[info] saving previous games '+str([g['game_id'] for g in self.game.previous_games]))
            for game in self.game.previous_games:
                self.history.write(game)    
            self.game.previous_games = []
            self.history.flush()
        if time.time() - self.log.lastflush  > period or period==0:
            self.log.flush()

In [ ]:
class BotWrap:
    def __init__(self, token, log):
        self.token = token
        self.bot = telegram.Bot(token)
        self.log = log
        self.last_update_id = 0
        
    def getUpdates(self):
        try:
            updates = self.bot.getUpdates(offset=self.last_update_id+1)
            if len(updates)>0:
                self.last_update_id = max(u.update_id for u in updates)
            return updates
        except Exception as e:
            self.log.write(format_exception(e))
            self.bot = telegram.Bot(self.token)
            return []
    
    def sendMessage(self, chat_id, answer):
        try:
            self.bot.sendMessage(chat_id, answer)
            return True
        except Exception as e:
            self.log.write(format_exception(e))
            return False
        
    def getMe(self):
        return self.bot.getMe()

In [ ]:
from datetime import datetime, timedelta
import time
from collections import defaultdict
import codecs, re, telegram, itertools, random

#imitate users and group chats
class TestBotWrap:
    def __init__(self, inputfile, log, delay=1):
        '''read all lines in file and use them to imitate many chats
        each line has format as follow:
        time:5 chat:-123399 user:Ivan message:hello, my dear Bot!!!\nРад снова с тобой поиграть'''
        self.log = log
        self.last_update_id = 0
        
        start_datetime = datetime.fromtimestamp(time.time()) + timedelta(seconds=delay)
        
        users = defaultdict(lambda: len(users)+random.randint(100000,999999)*10)
        chat_msgs_num = defaultdict(int)
        messages = []
        with codecs.open(inputfile, 'r', 'utf8') as f:
            for line in f:
                if line.strip() == "":
                    continue
                m = re.match(
                    r'time:(?P<time>\d+\.?\d*)\tchat:(?P<chat_id>-?(\d+))\tuser:(?P<user_name>[^\t]+)\tmessage:(?P<message>.*)$',
                    line)
                if m:
                    t = float(m.group('time'))
                    chat_id = int(m.group('chat_id'))
                    user_name = m.group('user_name')
                    user_id = users[(chat_id,user_name)]
                    message = re.sub(r'\\n', '\n',m.group('message')).strip()
                    user = telegram.User(user_id, user_name)
                    chat = telegram.GroupChat(chat_id, 'GroupChat_'+str(chat_id))
                    chat_msgs_num[chat_id] += 1
                    forward_date = start_datetime+timedelta(seconds=t)
                    msg = telegram.Message(chat_msgs_num[chat_id], user, forward_date, chat, text=message, forward_date=forward_date)
                    messages.append(msg)
                else:
                    print('can not parse '+line)
        messages = sorted(messages, key=lambda msg: msg.date)
        self.updates = []
        for msg in messages:
            self.updates.append(telegram.Update(len(self.updates), message=msg))
            
    def getUpdates(self):

        now = datetime.fromtimestamp(time.time())
        checkdate = lambda u: u.message.forward_date < now
        updates = list(itertools.takewhile(checkdate, self.updates[self.last_update_id:]))
        self.last_update_id += len(updates)
        return updates
       
    
    def sendMessage(self, chat_id, answer):
        return True
        
    def getMe(self):
        return telegram.User(12345, 'testBotWrap')
    
    def isStopped(self):
        return self.last_update_id >= len(self.updates)

In [ ]:
bot = TestBotWrap(r'test\test1.txt', Log(r'test\test1.log.txt'))
while not bot.isStopped():
    updates = bot.getUpdates()
    for u in updates:
        print (u.update_id, u.message.from_user.first_name, u.message.text)

In [ ]:
class Config:
    def __init__(self, **kvargs):
        for k,v in kvargs.items():
            self.__dict__[k] = v

In [ ]:
def get_bot_and_config():
    token = open("bot.token.txt").read().strip()
    bot = BotWrap(token, Log('telegram.log.txt'))
    print("me: "+bot.getMe().first_name)
    c = Config(**{'datadir': r'data\{0}', 
                'period_save_chat': 120, 
                'time_keep_chat': 60*60*2, 
                'period_life_message': 60*30,
                'period_common_stat': 120})
    return (bot,c)

In [ ]:
def get_test_bot_and_config():
    token = open("testbot.token.txt").read().strip() 
    testdir = r'test\output\test_telegram'
    bot = BotWrap(token, Log(testdir + r'\telegram.log.txt'))
    print("me: "+bot.getMe().first_name)
    c = Config(**{'datadir': testdir+ r'\data\{0}', 
                'period_save_chat': 15, 
                'time_keep_chat': 60, 
                'period_life_message': 30,
                'period_common_stat': 15})
    return (bot,c)

In [ ]:
import shutil
import os.path
def get_file_test_bot_and_config(testname):
    testfile = r'test\test'+str(testname)+'.txt'
    testdir =r'test\output\test'+str(testname)+'\\'
    if os.path.exists(testdir):
        shutil.rmtree(testdir, ignore_errors=True)
    bot = TestBotWrap(testfile, Log(testdir + r'telegram.log.txt'))
    print("me: "+bot.getMe().first_name)
    c = Config(**{'datadir': testdir+ r'data\{0}', 
                'period_save_chat': 15, 
                'time_keep_chat': 60, 
                'period_life_message': 30,
                'period_common_stat': 15})
    return (bot,c)

In [ ]:
import telegram
import time
from collections import defaultdict
import json
from datetime import timedelta

chats = {} # data for chats: games, saving, logs and etc.

speaker = get_speaker(r'locale\ru\phrases.json')
command_parser = get_command_parser(r'locale\ru\commands.json')

nouns = Vocabulary.read_definitions_from(r'locale\ru\vocab\noun.txt', saveas=lambda line: int(line[1]))
allwords = Vocabulary.read_from(r'locale\ru\vocab\all.txt')
defs = Vocabulary.read_multiple_definitions_from(r'locale\ru\vocab\ozhegov.hint.txt', saveas=lambda line: line[1])

task_factory = GameTaskFactory(nouns, allwords, defs)
       
(bot, config) = get_bot_and_config()

#общий лог - для статистики, ошибок, количества активных чатиков и т.д и т.п
log = bot.log
last = Config()
last.life_message = 0
last.stat_update = 0

def process_update(u):
    chat_id = u.message.chat.id
    user_id = u.message.from_user.id
    user_name = u.message.from_user.first_name

    if chat_id not in chats:
        chats[chat_id] = Chat(chat_id, lambda: WordGame(task_factory, speaker, command_parser), datadir=config.datadir)  
        log.write('OPEN NEW CHAT: '+str(chat_id))

    chat = chats[chat_id]
    chat.log.write('[user {0} {1}] {2}'.format(user_id, user_name, u.message.text))

    if user_id not in chat.game.players:
        chat.game.register_player(user_id, user_name);

    try:
        answer = chat.game.process(user_id, u.message.text)
    except Exception as e:
        log.write(format_exception(e))
        chat.log.write(format_exception(e))
        return False
    if answer is not None and answer != "":
        is_success = bot.sendMessage(chat_id, answer)
        chat.log.write('[bot] '+answer)
        chat.save(period = config.period_save_chat) #periodically save game and history
    return True
        
def update_common_stat():
    '''write common statistics and also clear some chats'''
    chat_ids = list(chats.keys())
    for chat_id in chat_ids:
        chat = chats[chat_id]
        if time.time() - chat.game.updated_at > config.time_keep_chat:
            try:
                chat.save(0) #force saving all data and log
            except Exception as e:
                log.write('Unexpected error in update_common_stat() Cant save chat '+str(chat_id)+': '+format_exception(e))
            del chats[chat_id]
    active_chats = '[INFO] Active chats in last '+str(timedelta(seconds=config.time_keep_chat))+': '+str(len(chats))
    log.write(active_chats)
    log.flush()
    last.stat_update = time.time()
    if time.time() - last.life_message > config.period_life_message:
        print(time.strftime("%Y.%m.%d %H:%M:%S ", time.localtime())+' in work')
        print(active_chats)
        last.life_message = time.time()

isRunning = True
while isRunning:
    try:
        updates = bot.getUpdates()
        for u in updates: 
            try:
                process_update(u)
            except Excetion as e:
                log.write('Unexpected error in process_update('+str(u)+'): '+format_exception(e))
                
        if time.time() - last.stat_update > config.period_common_stat:
            #иногда производим "чистку" и подсчет общей статистики
            update_common_stat()
            isRunning = not os.path.exists('stop.txt')
            
        time.sleep(1)
            
    except Exception as e:
        log.write("Unexpected error: "+format_exception(e))

for chat in chats.values():
    chat.save(0)
log.flush()
if not isRunning:
    print('Bot was stopped.')
    


In [ ]:
import codecs
nouns = dict()
with codecs.open(r'locale\ru\vocab\noun.full.txt', 'r', 'utf8') as f:
    for line in f:
        w = line.strip()
        if w != "":
            nouns[w] = 3
with codecs.open(r'locale\ru\vocab\noun.comm.txt', 'r', 'utf8') as f:
    for line in f:
        w = line.strip()
        if w != "":
            nouns[w] = 1
with codecs.open(r'locale\ru\vocab\dicts\mynouns.txt', 'r', 'utf8') as f:
    for line in f:
        w = line.strip()
        if w != "":
            if w not in nouns:
                nouns[w] = 1
with codecs.open(r'locale\ru\vocab\noun.txt', 'w+', 'utf8') as f:
    for w in sorted(nouns.keys()):
        f.write(w+'|'+str(nouns[w])+'\n')

In [ ]:
hint = Hint('anagram', 'blabla bla')
print(hint._export())
h = Hint._import(hint._export())
print(h._export())

In [ ]:
import telegram
help(telegram.Message)

In [ ]:
import datetime
import time


d = datetime.datetime.fromtimestamp(time.time())
delta = datetime.timedelta(seconds=60*60*2)
print(d)
print(delta)
print(d+delta)

print(time.strftime("%Y.%m.%d %H:%M:%S ", time.gmtime(0)))
print(time.strftime("%Y.%m.%d %H:%M:%S ", time.localtime()))